In [3]:
import yaml

# Define a configuração do dataset
config = {
    'path': '../data/yolo_dataset', # Caminho base que criamos no script anterior
    'train': 'train/images',
    'val': 'val/images',
    'kpt_shape': [8, 3], # 8 pontos, cada um com (x, y, visibilidade)
    'names': {
        0: 'cow'
    }
}

# Salva o arquivo YAML
with open('cow_pose.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("✅ Arquivo cow_pose.yaml criado com sucesso!")

✅ Arquivo cow_pose.yaml criado com sucesso!


In [4]:
import torch
from ultralytics import YOLO

# Verifica se a GTX 1650 está disponível
if torch.cuda.is_available():
    device = torch.device(0)
    print(f"✅ GPU Detectada: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM Total: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print("⚠️ GPU NÃO DETECTADA. O treino rodará na CPU.")

✅ GPU Detectada: NVIDIA GeForce GTX 1650
💾 VRAM Total: 3.63 GB


In [6]:
import pandas as pd
from pathlib import Path
import shutil
from PIL import Image
from tqdm import tqdm

def preparar_dataset_yolo(csv_path, img_root, output_dir, split_ratio=0.8):
    df = pd.read_csv(csv_path)
    output_path = Path(output_dir)
    
    # Estrutura padrão YOLO: images e labels para train e val
    for p in ['train/images', 'train/labels', 'val/images', 'val/labels']:
        (output_path / p).mkdir(parents=True, exist_ok=True)

    # Ordem fixa dos 8 pontos conforme seu CSV
    ordem_pontos = ['withers', 'back', 'hook up', 'hook down', 'hip', 'tail head', 'pin up', 'pin down']
    
    imagens = df['imagem'].unique().tolist() # Adicione o .tolist() aqui
    import numpy as np
    np.random.seed(42)
    np.random.shuffle(imagens)
    
    num_train = int(len(imagens) * split_ratio)
    
    print(f"📦 Convertendo {len(imagens)} imagens...")
    
    for i, img_nome in enumerate(tqdm(imagens)):
        folder = 'train' if i < num_train else 'val'
        
        # Localização da imagem original baseada na pasta do aluno
        info_aluno = df[df['imagem'] == img_nome]['aluno'].iloc[0]
        # Remove hash de ID se houver no nome do arquivo físico
        img_busca = img_nome.split('-')[-1]
        arquivos = list(Path(img_root).rglob(f"*{img_busca}*"))
        
        if not arquivos: continue
        img_origem = arquivos[0]
        
        with Image.open(img_origem) as img:
            w_img, h_img = img.size
        
        # Copia para a nova estrutura
        shutil.copy(img_origem, output_path / folder / 'images' / img_nome)
        
        # Cálculo das coordenadas para o YOLO (normalizadas de 0 a 1)
        pontos_img = df[df['imagem'] == img_nome]
        x_min, x_max = pontos_img['x'].min(), pontos_img['x'].max()
        y_min, y_max = pontos_img['y'].min(), pontos_img['y'].max()
        
        # Bounding Box (cx, cy, w, h)
        bw = (x_max - x_min + 10) / 100 
        bh = (y_max - y_min + 10) / 100
        cx = (x_min + x_max) / 2 / 100
        cy = (y_min + y_max) / 2 / 100
        
        # Formato: [classe] [cx] [cy] [w] [h] [k1_x] [k1_y] [k1_v] ...
        line = f"0 {cx:.6f} {cy:.6f} {bw:.6f} {bh:.6f}"
        
        for p_nome in ordem_pontos:
            p_data = pontos_img[pontos_img['ponto'] == p_nome]
            if not p_data.empty:
                px = p_data.iloc[0]['x'] / 100
                py = p_data.iloc[0]['y'] / 100
                # v=2 (visível), v=1 (oculto)
                v = 2 if p_data.iloc[0]['visibilidade'] == 'Visível' else 1
                line += f" {px:.6f} {py:.6f} {v}"
            else:
                line += " 0.0 0.0 0" # Ponto ausente
                
        with open(output_path / folder / 'labels' / img_nome.replace('.jpg', '.txt'), 'w') as f:
            f.write(line)

# Executa a conversão
preparar_dataset_yolo(
    csv_path='../data/processed/dataset_completo.csv',
    img_root='../data/raw',
    output_dir='../data/yolo_dataset'
)

📦 Convertendo 853 imagens...


100%|█████████████████████████████████████████| 853/853 [00:10<00:00, 85.14it/s]


In [7]:
import yaml
from ultralytics import YOLO

# 1. Gerar arquivo cow_pose.yaml
config = {
    'path': '../data/yolo_dataset', 
    'train': 'train/images',
    'val': 'val/images',
    'kpt_shape': [8, 3], # 8 keypoints: (x, y, visibility)
    'names': {0: 'cow'}
}

with open('cow_pose.yaml', 'w') as f:
    yaml.dump(config, f)

# 2. Carregar o modelo pré-treinado Nano (ideal para 4GB VRAM)
model = YOLO('yolov8n-pose.pt')

# 3. Iniciar Treino
# Reduzimos imgsz para 480 para garantir que caiba nos 3.63GB de VRAM
results = model.train(
    data='cow_pose.yaml',
    epochs=100,
    imgsz=480,
    batch=8,
    device=0, # Usa a GTX 1650 detectada
    name='yolov8_cattle_pose',
    augment=True
)

Ultralytics 8.4.15 🚀 Python-3.12.3 torch-2.10.0+cu128 CUDA:0 (NVIDIA GeForce GTX 1650, 3715MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cow_pose.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=480, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-pose.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=yolov8_cattle_pose, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 